In [ ]:
import napari
import numpy as np

from skimage import io
from skimage import data
from skimage.feature import SIFT, match_descriptors
from skimage.transform import AffineTransform, warp, estimate_transform
from skimage.measure import ransac
from skimage.color import rgb2gray

In [ ]:
# Create and run the napari viewer
viewer = napari.Viewer()
napari.run()

In [ ]:
# Load the demo data
image_original = data.skin()
image_warped = io.imread('skin_warped.png')

## Explore affine transforms

Affine transforms can be applied directly to napari layers. 

Run the cells below to see how an image layer can be transformed using different affine transform matrices.

In [ ]:
# Add the example image to the viewer
image_layer = viewer.add_image(image_original)

In [ ]:
# Define an itentity matrix
identity_transform = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
])

# Apply the transform to the image layer
image_layer.affine = identity_transform

In [ ]:
# Define a rotation matrix
rotation_transform = np.array([
    [ np.cos(np.radians(30)), np.sin(np.radians(30)), 0],
    [-np.sin(np.radians(30)), np.cos(np.radians(30)), 0],
    [0,                       0,                      1],
])

# Apply the transform to the image layer
image_layer.affine = rotation_transform

In [ ]:
# Define a scaling matrix
scaling_transform = np.array([
    [2, 0, 0],
    [0, 3, 0],
    [0, 0, 1],
])

# Apply the transform to the image layer
image_layer.affine = scaling_transform

In [ ]:
# Define a rotation matrix
translation_transform = np.array([
    [1, 0, 200],
    [0, 1, 400],
    [0, 0, 1],
])

# Apply the transform to the image layer
image_layer.affine = translation_transform

In [ ]:
# Combine the rotation and scaling transforms
combined_transform = rotation_transform @ scaling_transform

# Apply the transform to the image layer
image_layer.affine = combined_transform

In [ ]:
# Define an affine matrix
affine_transform = np.array([
    [1.09,  0.646, 79.6],
    [0.103, 0.966, 639],
    [0,     0,     1],
])

# Apply the transform to the image layer
image_layer.affine = affine_transform

## Manual image registration

An affine matrix to register two images can be found given two sets of matching points.

Matching points in both images can be selected with a points layer in napari.

Once the affine matrix is calculated, it can be applied to the moving image layer.

In [ ]:
viewer.layers.clear()

# Add the example image to the viewer
image_layer = viewer.add_image(image_original)

# Add the transformed image to the viewer
image_warped_layer = viewer.add_image(image_warped)

In [ ]:
# Add points layers to the viewer
moving_points = viewer.add_points(name='moving points', face_color='blue')
fixed_points = viewer.add_points(name='fixed points', face_color='red')

Before running the next cell, select matching points in both images using the `moving_points` and `fixed_points` layers. Set fixed points matching the original image, and moving points matching warped image.

In [ ]:
# Obtain the coordinates from the points layers
moving_coords = moving_points.data
fixed_coords = fixed_points.data

# Calculate the affine transform to convert from the moving layer to the fixed layer
affine_transform = estimate_transform('affine', moving_coords, fixed_coords)

print('Affine transform matrix:')
print(affine_transform.params)

In [ ]:
# Apply the transform to the moving image and points layers
image_warped_layer.affine = affine_transform.params
moving_points.affine = affine_transform.params

## Image registration with affinder
[affinder](https://github.com/jni/affinder) is a plugin that uses a similar method to find the affine matrix to register two images.

To use the plugin, open napari and go to Plugins->Install/Uninstall Plugins and search for 'affinder'.

## Image registration with SIFT
[Scale-Invariant Feature Transform (SIFT)](https://doi.org/10.1023/B:VISI.0000029664.99615.94) is a method to register two images using extracted keypoints and features. 

The approach has three steps:
1. Keypoints are extracted from distinctive points in both images (at corners, edges etc). At each keypoint, feature descriptors are also extracted.
2. Using the feature descriptors, matched points are found in both images.
3. The matched points are used to find the transform matrix

In [ ]:
viewer.layers.clear()

# Add the example image to the viewer
image_layer = viewer.add_image(image_original)

# Add the transformed image to the viewer
image_warped_layer = viewer.add_image(image_warped)

In [ ]:
# Extract keypoints and features from the fixed image
sift_fixed = SIFT()
sift_fixed.detect_and_extract(rgb2gray(image_original))
fixed_keypoints = sift_fixed.keypoints
fixed_descriptors = sift_fixed.descriptors

# Add the fixed keypoints to the viewer
viewer.add_points(fixed_keypoints, face_color='red')

In [ ]:
# Extract keypoints and features from the moving image
sift_moving = SIFT()
sift_moving.detect_and_extract(rgb2gray(image_warped))
moving_keypoints = sift_moving.keypoints
moving_descriptors = sift_moving.descriptors

# Add the moving keypoints to the viewer
viewer.add_points(moving_keypoints, face_color='blue')

In [ ]:
# Find matches between moving and fixed points
matches = match_descriptors(moving_descriptors, fixed_descriptors, cross_check=True)
moving_keypoints_matched = moving_keypoints[matches[:, 0]]
fixed_keypoints_matched = fixed_keypoints[matches[:, 1]]

# Add the matched points to the viewer
moving_points = viewer.add_points(moving_keypoints_matched, face_color='blue')
fixed_points = viewer.add_points(fixed_keypoints_matched, face_color='red')

In [ ]:
# Use ransac to estimate the affine transform
affine_transform, inliers = ransac(
    (moving_keypoints_matched, fixed_keypoints_matched),
    AffineTransform,
    min_samples=4,
    residual_threshold=2,
)

moving_points_inliers = viewer.add_points(
    moving_keypoints_matched[inliers], 
    name='moving_points_inliers', 
    face_color='blue')

fixed_points_inliers = viewer.add_points(
    fixed_keypoints_matched[inliers], 
    name='fixed_points_inliers', 
    face_color='red')

# Print the transformation matrix
print("Affine transform matrix:")
print(affine_transform.params)

In [ ]:
# Apply the transform to the transformed image layer
image_warped_layer.affine = affine_transform.params
moving_points_inliers.affine = affine_transform.params

So far, the affine matrices have only been applied to the napari layer which doesn't alter the data.

We can use the `warp` function to apply the transform directly to the image.

Scikit-image expects the transform to be given in (x, y) format whereas we have calculated it in (y, x) format using the numpy standard. It also expects the inverse transform that maps from the original image to the transformed image.

We first need to permute the transform and then calculate the inverse to obtain the correct format.


In [ ]:
# Permute the transform matrix
affine_transform_permuted = affine_transform.params[np.ix_([1, 0, 2], [1, 0, 2])]
affine_transform_inverse = np.linalg.inv(affine_transform_permuted)

# Compute the transformed image
image_aligned = warp(
    image_warped, 
    affine_transform_inverse,  # inverse transform in [x, y, z] order
    output_shape=image_original.shape,  # use the original image shape,
    preserve_range=True  # keep values between 0-255
)

# Convert to uint8 before saving
image_aligned = image_aligned.astype(np.uint8)

# Save the transformed image
io.imsave('skin_aligned.png', image_aligned)